In [1]:
from typing import List
import numpy as np
import imageio
import cv2
import copy
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def load_images(filenames: List) -> List:
    return [imageio.imread(filename) for filename in filenames]

In [3]:
def create_imgs_path(num_images: int):
    imgs_path = []
    for i in range(num_images):
        imgs_path.append(f"Images_calibration/Input/captured_image_{i}.jpg")
    return imgs_path

In [4]:
imgs_path = create_imgs_path(10)
imgs = load_images(imgs_path)



C:\Users\mtfra\AppData\Local\Temp\ipykernel_18660\283608422.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return [imageio.imread(filename) for filename in filenames]


In [5]:
# TODO Find corners with cv2.findChessboardCorners()
bad_img_idxs = []
corners = []
for i,img in enumerate(imgs):
    corners_img = cv2.findChessboardCorners(img,(7,7), None)
    if corners_img[0]:
        corners.append(corners_img)
    else:
        bad_img_idxs.append(i)

imgs = [img for idx, img in enumerate(imgs) if idx not in bad_img_idxs]

In [6]:
corners_copy = copy.deepcopy(corners)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# TODO To refine corner detections with cv2.cornerSubPix() you need to input greyscale images. Build a list containing greyscale images.
imgs_grey = []
for img in imgs:
    img_grey = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    imgs_grey.append(img_grey)

corners_refined = [cv2.cornerSubPix(i, cor[1], (7, 7), (-1, -1), criteria) if cor[0] else [] for i, cor in zip(imgs_grey, corners_copy)]

In [7]:
imgs_copy = copy.deepcopy(imgs)

In [8]:
len(imgs_copy)

8

In [9]:
# TODO Use cv2.drawChessboardCorners() to draw the cornes
for i in range(len(imgs_copy)):
    cv2.drawChessboardCorners(imgs_copy[i], (7,7), corners_refined[i], True)

In [10]:
# TODO Show images and save when needed
def show_image(name,img,key=500):
    cv2.imshow(f'{name}',img)
    cv2.waitKey(key)
    cv2.destroyAllWindows()

def write_image(path,img):
    cv2.imwrite(path,img)

In [11]:
path_inicial = "Images_calibration/output/"
if not os.path.exists(path_inicial):
    os.makedirs(path_inicial)
for i in range(len(imgs_copy)):
    complete_path = os.path.join(path_inicial,f"calibrated{i}.jpg")
    show_image(complete_path,imgs_copy[i])
    write_image(complete_path,imgs_copy[i])

In [12]:
# TODO Design the method. It should return a np.array with np.float32 elements
def get_chessboard_points(chessboard_shape, dx, dy):
    eje_x = chessboard_shape[1]
    eje_y = chessboard_shape[0] 
    coord_array = np.zeros((eje_x * eje_y, 3), dtype=np.float32)
    index_x = 0
    for j in range(eje_x):
        for i in range(eje_y):
            coord_array[index_x][0] = j*dy 
            coord_array[index_x][1] = i*dx 
            index_x += 1
    return coord_array

In [13]:
chessboard_points = get_chessboard_points((7, 7), 17, 17)

In [14]:
# Filter data and get only those with adequate detections
valid_corners = [cor[1] for cor in corners if cor[0]]
# Convert list to numpy array
valid_corners = np.asarray(valid_corners, dtype=np.float32)

In [15]:
# TODO
obj_points = [chessboard_points for _ in range(len(valid_corners))]

rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(obj_points, np.squeeze(valid_corners, axis=2), img_grey.shape[::-1], None, None)

# Obtain extrinsics
extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))

In [16]:
# Print outputs
print("Intrinsics:\n", intrinsics)
print("Distortion coefficients:\n", dist_coeffs)
print("Root mean squared reprojection error:\n", rms)

Intrinsics:
 [[1.43223718e+03 0.00000000e+00 6.72773648e+02]
 [0.00000000e+00 1.43400571e+03 3.61043530e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients:
 [[-1.97408932e-01  3.42824122e+00 -6.73726222e-03  3.08120953e-03
  -1.90945349e+01]]
Root mean squared reprojection error:
 1.536751060804123
